# Explicação rápida dos dados (resumo):

- Registros: clientes (customer_id) com variáveis demográficas, comportamentais, transacionais, datas, texto e target churn.
- Numéricas: age, income (forte skew + outliers), tenure_months, num_transactions, avg_transaction. 
    - skew se refere a desequilibrios ou distorções na distribuição.
    - outliers são valores extremos que podem distorcer análises estatísticas. Em termos gerais, são dados que fogem do padrão esperado.
- Categóricas: country, product, device (BR tem peso maior).
- Datas: signup_date, last_login — permitem criar recência, idade da conta, frequência.
- Texto: review_text (comentários curtos, útil para TF-IDF/embeddings). -> análise de sentimentos, tópicos.
- Target: churn (binário, desequilibrado — ~10% base). -> prever abandono.
- Problemas artificiais incluídos: missing (income, last_login, review_text), outliers extremos em income/avg_transaction, duplicatas, distribuição enviesada.
    - Esses problemas simulam desafios reais em dados de clientes.

# Tarefa proposta (prática de feature engineering + baseline):

### Objetivo: montar um pipeline reprodutível que faça engenharia de features, treine um modelo simples e reporte métricas.

> Passos mínimos (sugestão):

1. Exploração
--------------------------------------------------------------------------------------------------------------------------------------------------
> summary estatistico: refere-se a uma análise descritiva dos dados que inclui medidas como média, mediana, desvio padrão, valores mínimos e máximos, entre outros.
- usa-se: 
    - ```df.describe()``` no pandas.

> distribuições: refere-se à forma como os dados estão distribuídos para cada variável. 
- Isso pode incluir:
    - histogramas:
        - usa-se:
            - ````sns.histplot()```` do seaborn, ````plt.hist()```` do matplotlib.
    - boxplots:
        - usa-se:
            - ````sns.boxplot()```` do seaborn, ````plt.boxplot()```` do matplotlib.
    - KDE plots (Kernel Density Estimation):
        - usa-se:
            - ````sns.kdeplot()```` do seaborn.

> correlações: medem quanto uma variável se relaciona com outra.
    > A matriz de correlação ````df.corr()```` mostra valores entre -1 e 1, indicando a força e direção da relação.
- +1: correlação positiva perfeita (ambas crescem juntas)
- 1: correlação negativa perfeita (uma cresce, outra diminui)
- 0: sem relação linear clara
- usa-se:
    - ````sns.heatmap()```` do seaborn para visualizar a matriz de correlação.

> contagem por categoria: refere-se à contagem de ocorrências para cada categoria em variáveis categóricas.
- usa-se:
    - ````df['categoria'].value_counts()```` no pandas.

> percentuais de nulos: refere-se à proporção de valores ausentes em cada coluna do dataset.
- usa-se:
    - ````df.isnull().mean() * 100```` no pandas. -> lê-se como "percentual de valores nulos por coluna".


> checar duplicatas e formato de datas: refere-se à verificação de registros duplicados no dataset e à validação do formato das colunas de data.
- importa pois:
    - duplicatas podem ganhar peso indevido.
    - datas mal formatadas impedem cálculos corretos de recência, idade da conta, etc. 
        - exemplo: 
            - “2025/01/02”, “02-01-2025” e “01-02-25” podem representar coisas diferentes dependendo do padrão adotado (americano vs brasileiro).
            - converter signup_date/last_login para datetime.
- usa-se para duplicatas:

    ```python
     # para contar duplicatas.
    df.duplicated().sum()

     # para visualizar duplicatas.
    df[df.duplicated()]

    # para remover duplicatas.
    df.drop_duplicates()

    # para remover duplicatas parcialmente
    df = df.drop_duplicates(subset=['customer_id', 'signup_date'])
    ```

- usa-se para datas:
    ```python
    # Converter coluna de string para datetime
    df['data'] = pd.to_datetime(df['data'], errors='coerce', dayfirst=True)
    
    # Checar valores inválidos (que viraram NaT)
    df['data'].isna().sum()
    
    # Extrair partes úteis da data
    df['ano'] = df['data'].dt.year
    df['mes'] = df['data'].dt.month
    df['dia_semana'] = df['data'].dt.day_name()

    # signup_date/last_login
    df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce', dayfirst=True)
    df['last_login'] = pd.to_datetime(df['last_login'], errors='coerce', dayfirst=True)

    ```
----------------------------------------------------------------------------------------------------------------------------------------------------------------
2. Limpeza
    - remover duplicatas (total ou parcial por customer_id + signup_date).
    - converter signup_date/last_login para datetime.
    - criar coluna boolean indicando se last_login é nulo.
    - exemplo: 
    ```python
    df['no_login'] = df['last_login'].isna() # isna() retorna True para valores nulos
    ```
    <br><br>
3. Features temporais
    > recency = (today - last_login).days (usar um referência fixa para reprodutibilidade).
    - Conceito: Mede a quantidade de dias que se passaram desde a última interação (neste caso, o último login) do usuário até a data de hoje.
    - Interpreção: Um valor baixo indica que o usuário fez login recentemente e está engajado.Um valor alto indica que o usuário está inativo ou pode estar em risco de churn (abandono).
    uso: 
    ```python
    df['recency'] = (pd.Timestamp("2025-10-01") - df['last_login']).dt.days # exemplo com data fixa
    ```
    <br><br>
    > account_age_days = (today - signup_date).days.
    - Conceito: Mede a idade total da conta do usuário em dias, desde a data de cadastro (signup_date) até hoje.
    - Interpretação: Um valor alto indica que o usuário é um cliente antigo, o que pode sugerir lealdade. Um valor baixo indica um cliente novo, que pode estar em fase de avaliação do serviço.
    uso: 
    ```python
    df['account_age_days'] = (pd.Timestamp("2025-10-01") - df['signup_date']).dt.days # exemplo com data fixa
    ```
    <br><br>
    > activity_rate = num_transactions / max(1, account_age_days/30).
    - Conceito: Mede a frequência média de transações do usuário por mês, normalizando pelo tempo que a conta está ativa.
    - Detalhes da fórmula:
        - num_transactions: O número total de transações feitas pelo usuário.
        - account_age_days/30: Converte a idade da conta em dias para a idade da conta em meses.
        max(1, ...): É uma técnica de proteção contra divisão por zero ou por um período de tempo muito pequeno. Garante que o denominador seja pelo menos 1, o que é crucial para contas criadas há menos de um mês (para evitar uma taxa de atividade inflacionada ou divisão por zero).
    - Interpretação: Um valor alto indica um usuário ativo que realiza transações regularmente. Um valor baixo pode indicar um usuário inativo ou com baixo engajamento.
    uso: 
    ```python
    # Calcula a idade em meses, garantindo que o divisor seja no mínimo 1
    age_months = df_usuarios['account_age_days'] / 30
    divisor = np.maximum(1, age_months)

    df_usuarios['activity_rate'] = df_usuarios['num_transactions'] / divisor
    ```

4. Tratamento de missing
    > income: Preencher (os valores ausentes) com a mediana e criar uma flag chamada is_income_missing.
    > last_login: Preencher os valores ausentes usando signup_date (ou um valor que represente grande recency) e criar uma flag indicando que o valor foi imputado.
    > review_text: Substituir valores ausentes ou vazios por uma string vazia ("") e criar uma coluna indicando que o valor original estava faltando.

5. Outliers
    > Outliers são valores muito altos ou muito baixos que fogem ao padrão da maioria dos dados.
    > truncamento (winsorize) ou log-transform para income e avg_transaction; criar versão transformada.
        - truncamento (winsorize): É uma técnica que limita os valores extremos a um certo percentil. Em vez de remover outliers, você os “trunca” nos limites superior e inferior.
            - uso:
            ```python
            from scipy.stats.mstats import winsorize
            df['income_winsorized'] = winsorize(df['income'], limits=[0.01, 0.01])
            ```
        - log-transform: Aplica o logaritmo nos valores — útil para variáveis altamente assimétricas (skewed), como renda ou gasto médio.
            - uso: 
            ```python
            import numpy as np
            df['income_log'] = np.log1p(df['income'])
            df['avg_transaction_log'] = np.log1p(df['avg_transaction'])
            ```
6. Encoding
    > Endcoding é o processo de transformar variáveis categóricas ou texto em formatos numéricos que modelos de machine learning podem entender.
    - categóricas: One-Hot para device/product; target/mean-encoding ou frequency-encoding para country (avaliar leakage).
        - One-Hot Encoding: Cria uma nova coluna binária para cada categoria.
            - uso:
            ```python
            import pandas as pd
            pd.get_dummies(df, columns=['device', 'product'])
            ```
        - Target/Mean-Encoding: Substitui cada categoria pelo valor médio da variável-alvo (target) para essa categoria.
            - uso:
                ```python
                mean_target = df.groupby('country')['churn'].mean()
                df['country_mean_encoded'] = df['country'].map(mean_target)
                ```
        - Frequency-Encoding: Substitui cada categoria pela frequência (contagem) dessa categoria no dataset.
            - uso:
            ```python
            freq = df['country'].value_counts()
            df['country_freq_encoded'] = df['country'].map(freq)
            ```
    - texto: TF-IDF (unigram/bigram) limitando vocab (top k) ou usar hash vectorizer.
        - TF-IDF (Term Frequency - Inverse Document Frequency): Transforma o texto em vetores numéricos, com base na importância das palavras.
            - Palavras muito frequentes no texto → peso maior
            - Mas se aparecem em todos os textos → peso menor
                - uso:
                ```python
                from sklearn.feature_extraction.text import TfidfVectorizer

                vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
                # ngram_range=(1,2) → considera unigramas (1 palavra) e bigramas (pares de palavras).
                # max_features=500 → limita o vocabulário às 500 palavras mais relevantes.
                X_tfidf = vectorizer.fit_transform(df['review_text'])
                ```
        - Hash Vectorizer: Similar ao TF-IDF, mas usa uma função de hash para mapear palavras em um espaço de características fixo, economizando memória.
            - uso:
            ```python
            from sklearn.feature_extraction.text import HashingVectorizer

            vectorizer = HashingVectorizer(n_features=5000, ngram_range=(1, 2))
            X_hashed = vectorizer.fit_transform(df['review_text'])
            ```
7. Pipeline e seleção
    - Pipeline: refere-se a uma sequência de etapas de processamento de dados e modelagem que são encadeadas juntas para formar um fluxo de trabalho completo.
    - Seleção: refere-se ao processo de escolher um subconjunto de features (variáveis) que são mais relevantes para o modelo, com o objetivo de melhorar o desempenho e reduzir a complexidade.
    - montar sklearn ColumnTransformer + Pipeline (imputer, scaler, encoders, text vect).
        - Esses dois componentes são a espinha dorsal de um pipeline de ML bem estruturado.
        - ColumnTransformer: Permite aplicar transformações diferentes para colunas diferentes no mesmo dataset.
        - uso: [ColumnTransformer Example](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)
        - Pipeline: Encadeia várias etapas de processamento e modelagem em uma única entidade.
        - uso: [Pipeline Example](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
    - reduzir dimensionalidade texto com TruncatedSVD se usar TF-IDF.
        - TruncatedSVD (Singular Value Decomposition truncada): É uma técnica de redução de dimensionalidade que é especialmente útil para dados esparsos, como os gerados por TF-IDF.
        - uso:
        ```python
        from sklearn.decomposition import TruncatedSVD

        svd = TruncatedSVD(n_components=100)  # Reduz para 100 dimensões
        X_reduced = svd.fit_transform(X_tfidf)
        ```
8. Entrega dos features
    - salvar features processadas: df_features.to_csv('datasets/processed_features.csv') ou pickle
    - documentar no notebook as decisões de imputação e outliers
<br>
### Dicas rápidas de implementação:
- use sklearn.compose.ColumnTransformer, sklearn.pipeline.Pipeline.
- para texto: sklearn.feature_extraction.text.TfidfVectorizer + TruncatedSVD.
- para avaliação temporal: sort by signup_date e use TimeSeriesSplit ou cutoff manual.
- fixe "today" com uma data constante (ex.: pd.Timestamp("2025-10-01")) para reprodutibilidade.